In [23]:
from os.path import join as pjoin
from os import listdir as lsdir
from obspy.geodetics import kilometer2degrees
from obspy.taup import TauPyModel
from SS_MTI import Gradient, PhaseTracer
import Create_Vmod

# Save directory

In [19]:
save_path = "/home/nienke/Documents/Research/SS_MTI/External_packages/Test_reflectivity/m0_gradient_descent/"

# Fixed parameters

In [3]:
bin_path = "/home/nienke/Documents/Research/SS_MTI/External_packages/reflectivity_Mars/SRC/test/crfl_sac"
if not lsdir(save_path):
    subprocess.call(f"scp {bin_path} .", shell=True, cwd=save_path)

In [16]:
src_depth = 20.0
epi_in_km = 1774.7380
epi = kilometer2degrees(epi_in_km, radius=3389.5)
baz = 0.

dt = 0.025

phases = ["P", "S", "P", "S", "S"]
comps = ["Z", "T", "R", "Z", "R"]
t_pres = [1, 1, 1, 1, 1]
t_posts = [30, 30, 30, 30, 30]
ylims = [1e-9, 1e-9, 2e-9, 3e-9, 2e-9]

fmin = 0.2
fmax = 0.6
zerophase = False

# Observed data

In [17]:
path_observed = (
    "/home/nienke/Documents/Research/SS_MTI/External_packages/Test_reflectivity/obs/"
)
npz_file = "/home/nienke/Documents/Research/Data/npz_files/TAYAK.npz"
st_obs = Gradient.read_refl_mseeds(path=path_observed)
Taup = TauPyModel(npz_file)
obs_tts = [PhaseTracer.get_traveltime(Taup, phase, src_depth, epi) for phase in phases]
st_obs_w, st_obs_full, s_obs = Gradient.window(
    st_obs, phases, comps, obs_tts, t_pres, t_posts, fmin, fmax, zerophase,
)

# Start values

In [ ]:
bm_start_model = "/home/nienke/Documents/Research/Data/MTI/MT_vs_STR/bm_models/TAYAK.bm"
m_rr = 0.2
m_tt = 0.8
m_pp = 0.0
m_rt = 0.0
m_rp = 0.0
m_tp = 0.0
focal_mech = [m_rr, m_tt, m_pp, m_rt, m_rp, m_tp]
Moho_d = 77.3680

Create_Vmod.create_dat_file(
    src_depth, focal_mech, epi_in_km, baz, dt, save_path, bm_start_model,
)

In [ ]:
m0 = np.hstack((focal_mech,Moho_d))

# Line search

In [ ]:
src_str = _Gradient.SRC_STR(
    binary_file_path=bin_path,
    path_to_dat=save_path,
    phases=phases,
    components=comps,
    t_pres=t_pres,
    t_posts=t_posts,
    depth=True,
    vpvs=False,
    fmin=fmin,
    fmax=fmax,
    dt=dt,
    baz=baz,
    sigmas=sigmas,
    zerophase=zerophase,
    plot=True,
    st_obs_full=st_obs_full,
    tt_obs=obs_tts,
    ylims=ylims,
)